In [ ]:

import weaviate
import weaviate.classes.config as wvcc
from weaviate.classes.config import Configure

from ai_journal.storage import example_data_location
from ai_journal.text import read_and_chunk_files

client = weaviate.connect_to_local(host="localhost")

In [ ]:
client.collections.delete_all()

In [ ]:
collection = client.collections.create(
    name="WeaviateJournalChunk",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["title"],
            # If using Docker, use this to contact your local Ollama instance
            api_endpoint="http://host.docker.internal:11434",
            model="llama3:8b",  # The model to use, e.g. "nomic-embed-text"
        )
    ],
    generative_config=Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434"
    ),
    properties=[
        wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
        wvcc.Property(name="author", data_type=wvcc.DataType.TEXT),
    ],
)

In [ ]:
# Example usage
journal_chunks = read_and_chunk_files(example_data_location)

In [ ]:
len(journal_chunks)

journal_chunks[0]

In [ ]:
journal_collection = client.collections.get("WeaviateJournalChunk")

for idx, journal_chunk in enumerate(journal_chunks):
    upload = journal_collection.data.insert(properties={"content": journal_chunk})

print(f"Uploaded {idx} journal chunks.")